In [10]:
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Union, Optional
from tqdm import tqdm
from sklearn.metrics import ndcg_score
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from sentence_transformers import CrossEncoder

In [11]:
# Абстрактный класс для моделей эмбеддинга
class EmbeddingModel(ABC):
    @abstractmethod
    def compute_embeddings(self, texts: List[str]) -> Union[List[List[float]], np.ndarray]:
        """Вычисляет эмбеддинги для списка текстов."""
        pass
    
    @abstractmethod
    def compute_single_embedding(self, text: str) -> Union[List[float], np.ndarray]:
        """Вычисляет эмбеддинг для одного текста."""
        pass
    
    @abstractmethod
    def get_vector_size(self) -> int:
        """Возвращает размерность векторов для модели."""
        pass
    
    @abstractmethod
    def get_model_name(self) -> str:
        """Возвращает имя модели для отчетов."""
        pass

# Реализация для SentenceTransformer
class SentenceTransformerModel(EmbeddingModel):
    def __init__(self, model_name: str = 'BAAI/bge-large-en-v1.5'):
        self.model_name = model_name
        self.model = SentenceTransformer(model_name)
    
    def compute_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, normalize_embeddings=True)
        return [embedding.tolist() for embedding in embeddings]
    
    def compute_single_embedding(self, text: str) -> List[float]:
        embedding = self.model.encode(text, normalize_embeddings=True)
        return embedding.tolist()
    
    def get_vector_size(self) -> int:
        return self.model.get_sentence_embedding_dimension()
    
    def get_model_name(self) -> str:
        return f"SentenceTransformer-{self.model_name.split('/')[-1]}"

# Реализация для BM25
class BM25Model(EmbeddingModel):
    def __init__(self):
        self.corpus = None
        self.bm25 = None
        self.vector_size = 768  # Фиктивное значение для совместимости с Qdrant
        # Подготовка для токенизации и стемминга
        self.stemmer = PorterStemmer()
        try:
            nltk.data.find('tokenizers/punkt')
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('punkt')
            nltk.download('stopwords')
        self.stop_words = set(stopwords.words('english'))
    
    def preprocess_text(self, text: str) -> List[str]:
        """Предобработка текста: токенизация, удаление стоп-слов, стемминг."""
        tokens = word_tokenize(text.lower())
        tokens = [self.stemmer.stem(token) for token in tokens if token.isalnum() and token not in self.stop_words]
        return tokens
    
    def initialize_corpus(self, texts: List[str]) -> None:
        """Инициализация индекса BM25 на основе корпуса документов."""
        processed_corpus = [self.preprocess_text(text) for text in texts]
        self.corpus = texts  # Сохраняем оригинальные тексты
        self.bm25 = BM25Okapi(processed_corpus)
    
    def compute_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Для BM25 возвращаем фиктивные эмбеддинги.
        В реальном использовании будем напрямую использовать BM25 для поиска.
        """
        if self.corpus is None:
            self.initialize_corpus(texts)
        
        # Возвращаем фиктивные эмбеддинги для совместимости с API
        return np.random.rand(len(texts), self.vector_size)
    
    def compute_single_embedding(self, text: str) -> np.ndarray:
        """
        Для BM25 эмбеддинг не используется.
        Возвращаем фиктивный эмбеддинг для совместимости с API.
        """
        return np.random.rand(self.vector_size)
    
    def get_vector_size(self) -> int:
        return self.vector_size
    
    def get_model_name(self) -> str:
        return "BM25"
    
    def search(self, query: str, limit: int = 6) -> List[Dict[str, Any]]:
        """Поиск схожих документов с помощью BM25."""
        if self.bm25 is None:
            raise ValueError("BM25 не инициализирован. Сначала вызовите compute_embeddings с корпусом документов.")
        
        query_tokens = self.preprocess_text(query)
        scores = self.bm25.get_scores(query_tokens)
        
        # Получаем индексы документов, отсортированных по убыванию релевантности
        top_indices = np.argsort(scores)[::-1][:limit]
        
        # Формируем результаты в том же формате, что и для других моделей
        results = []
        for i in top_indices:
            if scores[i] > 0:  # Добавляем только документы с ненулевой релевантностью
                results.append({
                    'id': i,
                    'text': self.corpus[i],
                    'score': float(scores[i])
                })
        
        return results

In [12]:
# Абстрактный класс для реранкеров
class Reranker(ABC):
    @abstractmethod
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = None) -> List[Dict[str, Any]]:
        """Переранжирует документы на основе запроса."""
        pass
    
    @abstractmethod
    def get_reranker_name(self) -> str:
        """Возвращает имя реранкера."""
        pass

# Реализация без реранкера (проходной реранкер)
class NoReranker(Reranker):
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = None) -> List[Dict[str, Any]]:
        """Возвращает документы без изменения порядка."""
        if top_k is not None and top_k < len(documents):
            return documents[:top_k]
        return documents
    
    def get_reranker_name(self) -> str:
        return "No-Reranker"
    
# Реранкер на основе CrossEncoder
class CrossEncoderReranker(Reranker):
    def __init__(self, model_name: str):
        self.model_name = model_name
        
        # Настройки для токенизатора перед созданием модели
        self.max_length = 512  # Стандартный предел для большинства моделей
        
        # Создаем модель с указанием максимальной длины для токенизатора
        self.model = CrossEncoder(
            model_name,
            max_length=self.max_length,  # Устанавливаем максимальную длину при инициализации
            device=None  # Автоматический выбор устройства
        )
    
    def truncate_text(self, text: str, max_chars: int = 1500) -> str:
        """Обрезает текст до заданного количества символов."""
        if len(text) <= max_chars:
            return text
        return text[:max_chars] + "..."
    
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = None) -> List[Dict[str, Any]]:
        """Переранжирует документы с использованием CrossEncoder."""
        if not documents:
            return []
        
        # Подготовка пар (запрос, документ) для оценки, обрезая длинные тексты
        pairs = []
        for doc in documents:
            # Обрезаем длинные документы, чтобы избежать проблем с токенизацией
            truncated_text = self.truncate_text(doc['text'])
            pairs.append((query, truncated_text))
        
        # Получение оценок - без указания max_length, т.к. он уже указан при инициализации
        scores = self.model.predict(
            pairs,
            show_progress_bar=False
        )
        
        # Обновление оценок в документах
        for i, score in enumerate(scores):
            documents[i]['rerank_score'] = float(score)
        
        # Сортировка по новым оценкам
        reranked_docs = sorted(documents, key=lambda x: x['rerank_score'], reverse=True)
        
        # Ограничение количества результатов, если нужно
        if top_k is not None and top_k < len(reranked_docs):
            return reranked_docs[:top_k]
        return reranked_docs
    
    def get_reranker_name(self) -> str:
        return f"CrossEncoder-{self.model_name.split('/')[-1]}"

# Реранкер на основе HuggingFace моделей
class HuggingFaceReranker(Reranker):
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        # Определяем максимальную длину для токенизатора
        self.max_length = min(
            self.tokenizer.model_max_length if hasattr(self.tokenizer, "model_max_length") else 512,
            512  # Установим верхний предел в 512 токенов
        )
    
    def truncate_text(self, text: str, max_chars: int = 1500) -> str:
        """Обрезает текст до заданного количества символов."""
        if len(text) <= max_chars:
            return text
        return text[:max_chars] + "..."
    
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = None) -> List[Dict[str, Any]]:
        """Переранжирует документы с использованием HuggingFace модели."""
        if not documents:
            return []
        
        reranked_docs = []
        
        # Обрабатываем документы по одному для экономии памяти
        for doc in documents:
            # Обрезаем текст, чтобы избежать проблем с длинными документами
            truncated_text = self.truncate_text(doc['text'])
            
            # Токенизация пары запрос-документ
            inputs = self.tokenizer(
                query, 
                truncated_text, 
                padding=True, 
                truncation=True, 
                max_length=self.max_length, 
                return_tensors="pt"
            )
            
            # Перемещаем входные данные на GPU, если доступно
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            # Получаем оценку из модели
            with torch.no_grad():
                outputs = self.model(**inputs)
                scores = outputs.logits
                
                # Обработка в зависимости от формата выходных данных модели
                if scores.shape[1] == 1:  # Одна оценка (регрессия)
                    score = scores.item()
                else:  # Бинарная классификация (релевантный/нерелевантный)
                    score = torch.softmax(scores, dim=1)[:, 1].item()
            
            # Создаем копию документа с обновленной оценкой
            doc_copy = doc.copy()
            doc_copy['rerank_score'] = score
            reranked_docs.append(doc_copy)
        
        # Сортировка по новым оценкам
        reranked_docs = sorted(reranked_docs, key=lambda x: x['rerank_score'], reverse=True)
        
        # Ограничение количества результатов, если нужно
        if top_k is not None and top_k < len(reranked_docs):
            return reranked_docs[:top_k]
        return reranked_docs
    
    def get_reranker_name(self) -> str:
        return f"HF-{self.model_name.split('/')[-1]}"
    
# Специализированный реранкер для nboost/pt-tinybert-msmarco
class TinyBertReranker(Reranker):
    def __init__(self, model_name: str = 'nboost/pt-tinybert-msmarco'):
        self.model_name = model_name
        
        # Загружаем модель и токенизатор напрямую из HuggingFace
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        self.max_length = 512
    
    def truncate_text(self, text: str, max_chars: int = 1500) -> str:
        """Обрезает текст до заданного количества символов."""
        if len(text) <= max_chars:
            return text
        return text[:max_chars] + "..."
    
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = None) -> List[Dict[str, Any]]:
        """Переранжирует документы с использованием TinyBERT напрямую."""
        if not documents:
            return []
        
        reranked_docs = []
        
        for doc in documents:
            try:
                # Обрезаем текст
                truncated_text = self.truncate_text(doc['text'])
                
                # Токенизация
                inputs = self.tokenizer(
                    query,
                    truncated_text,
                    padding=True,
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt"
                )
                
                # Перемещаем на устройство
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                
                # Получаем оценку
                with torch.no_grad():
                    outputs = self.model(**inputs)
                    logits = outputs.logits
                    
                    # TinyBERT для MS MARCO обычно возвращает один скор
                    score = float(logits[0][0].cpu().numpy())
                
                # Создаем копию с оценкой
                doc_copy = doc.copy()
                doc_copy['rerank_score'] = score
                reranked_docs.append(doc_copy)
            
            except Exception as e:
                print(f"Ошибка при ранжировании с TinyBERT: {e}")
                # Добавляем с нулевой оценкой
                doc_copy = doc.copy()
                doc_copy['rerank_score'] = 0.0
                reranked_docs.append(doc_copy)
        
        # Сортировка
        reranked_docs = sorted(reranked_docs, key=lambda x: x['rerank_score'], reverse=True)
        
        # Ограничение количества
        if top_k is not None and top_k < len(reranked_docs):
            return reranked_docs[:top_k]
        return reranked_docs
    
    def get_reranker_name(self) -> str:
        return "TinyBERT-MSMARCO"

In [13]:
# Модифицированный класс QdrantManager для поддержки разных моделей
class QdrantManager:
    """Класс для работы с базой Qdrant."""
    
    def __init__(self, embedding_model: EmbeddingModel = None):
        self.client = QdrantClient(":memory:")
        self.collection_name = "documentation"
        self.embedding_model = embedding_model
        # Для сохранения оригинальных текстов (нужно для BM25)
        self.corpus = []
        self.id_to_idx = {}  # Отображение id -> индекс в corpus
    
    def set_embedding_model(self, embedding_model: EmbeddingModel) -> None:
        """Устанавливает модель для эмбеддинга."""
        self.embedding_model = embedding_model
    
    def initialize_collection(self, vector_size: int = None):
        """Инициализирует коллекцию в Qdrant."""
        if vector_size is None and self.embedding_model is not None:
            vector_size = self.embedding_model.get_vector_size()
        
        # Удаление существующей коллекции, если она есть
        try:
            self.client.delete_collection(collection_name=self.collection_name)
        except Exception:
            pass
        
        # Создание новой коллекции
        self.client.create_collection(
            collection_name=self.collection_name,
            vectors_config=models.VectorParams(
                size=vector_size,
                distance=models.Distance.COSINE
            )
        )
        
        # Очистка корпуса при инициализации новой коллекции
        self.corpus = []
        self.id_to_idx = {}
    
    def upsert_batch(self, id_offset: int, vectors: List[List[float]], 
                    payloads: List[Dict[str, Any]]) -> None:
        """Добавляет партию данных в Qdrant."""
        # Сохраняем тексты в корпусе (для BM25)
        for idx, payload in enumerate(payloads):
            corpus_idx = len(self.corpus)
            self.corpus.append(payload['text'])
            self.id_to_idx[idx + id_offset] = corpus_idx
        
        # Добавляем в Qdrant
        points = [
            models.PointStruct(
                id=idx + id_offset,
                vector=vector,
                payload=payload
            )
            for idx, (vector, payload) in enumerate(zip(vectors, payloads))
        ]
        
        self.client.upsert(
            collection_name=self.collection_name,
            points=points
        )
    
    def search(self, query: str, limit: int = 6) -> List[Dict[str, Any]]:
        """Выполняет поиск похожих документов."""
        # Для BM25 используем прямой поиск
        if isinstance(self.embedding_model, BM25Model):
            return self.embedding_model.search(query, limit=limit)
        
        # Для других моделей используем Qdrant
        query_vector = self.embedding_model.compute_single_embedding(query)
        search_result = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=limit
        )
        
        return [
            {
                'text': hit.payload['text'],
                'id': hit.payload['id'],
                'score': hit.score
            }
            for hit in search_result
        ]

In [14]:
class MetricsCalculator:
    """Класс для расчета метрик оценки качества поиска."""
    
    @staticmethod
    def calculate_recall_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает Recall@k для одного запроса."""
        if not relevant_ids:
            return 0.0
        
        relevant_retrieved = set(relevant_ids).intersection(set(retrieved_ids[:k]))
        return len(relevant_retrieved) / len(relevant_ids)
    
    @staticmethod
    def calculate_precision_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает Precision@k для одного запроса."""
        if k == 0 or not retrieved_ids:
            return 0.0
        
        relevant_retrieved = set(relevant_ids).intersection(set(retrieved_ids[:k]))
        return len(relevant_retrieved) / min(k, len(retrieved_ids))
    
    @staticmethod
    def calculate_mrr_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает MRR@k (Mean Reciprocal Rank) для одного запроса."""
        if not relevant_ids or not retrieved_ids:
            return 0.0
        
        for i, doc_id in enumerate(retrieved_ids[:k]):
            if doc_id in relevant_ids:
                return 1.0 / (i + 1)
        return 0.0
    
    @staticmethod
    def calculate_ndcg_at_k(relevant_ids: List[int], retrieved_ids: List[int], k: int) -> float:
        """Рассчитывает nDCG@k для одного запроса вручную."""
        if not relevant_ids or not retrieved_ids or k <= 0:
            return 0.0
        
        # Обрезаем список до k элементов
        retrieved_ids_k = retrieved_ids[:k]
        
        # Создаем вектор релевантности (1 для релевантных документов, 0 для нерелевантных)
        relevance = [1.0 if doc_id in relevant_ids else 0.0 for doc_id in retrieved_ids_k]
        
        # Если нет релевантных документов среди извлеченных, возвращаем 0
        if sum(relevance) == 0:
            return 0.0
        
        # Вычисляем DCG (Discounted Cumulative Gain)
        dcg = 0.0
        for i, rel in enumerate(relevance):
            # Используем формулу DCG = rel_1 + rel_2/log2(2+1) + rel_3/log2(3+1) + ...
            if rel > 0:
                dcg += rel / np.log2(i + 2)  # +2 потому что индексация с 0, и log2(1)=0
        
        # Вычисляем идеальный DCG (IDCG)
        # В идеальном случае релевантные документы находятся в начале списка
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            if rel > 0:
                idcg += rel / np.log2(i + 2)
        
        # nDCG = DCG / IDCG
        return dcg / idcg if idcg > 0 else 0.0
    
    @staticmethod
    def compute_average_metrics(metrics_list: List[Dict[str, float]]) -> Dict[str, float]:
        """Вычисляет средние значения метрик по всем запросам."""
        if not metrics_list:
            return {}
        
        # Инициализируем результирующий словарь
        result = {}
        
        # Собираем все ключи метрик
        all_keys = set()
        for metrics in metrics_list:
            all_keys.update(metrics.keys())
        
        # Вычисляем среднее значение для каждой метрики
        for key in all_keys:
            values = [metrics.get(key, 0.0) for metrics in metrics_list]
            result[key] = sum(values) / len(metrics_list)
        
        return result

In [15]:
class DocumentationQA:
    """Главный класс, объединяющий все компоненты системы."""
    
    def __init__(self, embedding_model: Optional[EmbeddingModel] = None, reranker: Optional[Reranker] = None):
        # Если модель не передана, используем SentenceTransformer по умолчанию
        self.embedding_model = embedding_model or SentenceTransformerModel()
        # Если реранкер не передан, используем NoReranker
        self.reranker = reranker or NoReranker()
        self.qdrant_manager = QdrantManager(self.embedding_model)
        self.metrics_calculator = MetricsCalculator()
        self.is_initialized = False
        self.df = None
        self.section_id_map = {}  # Для хранения мапинга section_content -> id
    
    def set_embedding_model(self, embedding_model: EmbeddingModel) -> None:
        """Устанавливает модель для эмбеддинга и сбрасывает инициализацию."""
        self.embedding_model = embedding_model
        self.qdrant_manager.set_embedding_model(embedding_model)
        self.is_initialized = False
    
    def set_reranker(self, reranker: Reranker) -> None:
        """Устанавливает реранкер."""
        self.reranker = reranker
    
    def load_data(self, file_path: str = 'qdrant_documentation_dataset.csv'):
        """Загружает данные из CSV файла."""
        self.df = pd.read_csv(file_path)
        print(f"Загружено {len(self.df)} записей из датасета.")
    
    def initialize_database(self):
        """Инициализирует базу данных, добавляя чанки из датасета в Qdrant."""
        if self.is_initialized:
            return
        
        if self.df is None:
            raise ValueError("Данные не загружены. Вызовите метод load_data() перед инициализацией базы.")
        
        # Получаем уникальные чанки контента
        sections = self.df['section_content'].unique()
        print(f"Найдено {len(sections)} уникальных чанков для индексации.")
        
        # Создаем мапинг section_content -> id
        for idx, section in enumerate(sections):
            self.section_id_map[section] = idx
        
        # Инициализация коллекции Qdrant
        vector_size = self.embedding_model.get_vector_size()
        self.qdrant_manager.initialize_collection(vector_size=vector_size)
        
        # Специальная обработка для BM25
        if isinstance(self.embedding_model, BM25Model):
            self.embedding_model.initialize_corpus(sections)
        
        # Вычисление эмбеддингов и добавление в Qdrant
        batch_size = 100
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i + batch_size]
            
            # Вычисление эмбеддингов
            print(f"Вычисление эмбеддингов для батча {i//batch_size + 1}/{(len(sections) - 1)//batch_size + 1}")
            embeddings = self.embedding_model.compute_embeddings(batch)
            
            # Подготовка payload
            payloads = [
                {
                    'text': section,
                    'id': self.section_id_map[section]
                }
                for section in batch
            ]
            
            # Добавление в Qdrant
            self.qdrant_manager.upsert_batch(i, embeddings, payloads)
        
        self.is_initialized = True
        print(f"База данных успешно инициализирована с моделью {self.embedding_model.get_model_name()}.")
    
    def search_similar_sections(self, query: str, top_k: int = 6, first_stage_k: int = None) -> List[Dict[str, Any]]:
        """Ищет чанки, похожие на запрос, с применением реранкинга."""
        # Определяем, сколько документов получить на первом этапе
        if first_stage_k is None:
            # По умолчанию берем больше документов для реранкинга, если используется реранкер
            first_stage_k = top_k * 3 if not isinstance(self.reranker, NoReranker) else top_k
        
        # Поиск через менеджер Qdrant (первый этап)
        first_stage_results = self.qdrant_manager.search(query, limit=first_stage_k)
        
        # Применяем реранкинг (второй этап)
        reranked_results = self.reranker.rerank(query, first_stage_results, top_k=top_k)
        
        return reranked_results
    
    def evaluate_model(self, k_values: List[int] = [1, 4, 6], first_stage_k: int = None) -> Dict[str, float]:
        """Оценивает модель по различным метрикам."""
        if not self.is_initialized:
            self.initialize_database()
        
        all_metrics = []
        model_name = self.embedding_model.get_model_name()
        reranker_name = self.reranker.get_reranker_name()
        
        print(f"Оценка модели: {model_name} с реранкером: {reranker_name}")
        
        # Перебираем все вопросы в датасете
        for idx, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Оценка модели"):
            question = row['question']
            relevant_section = row['section_content']
            relevant_id = self.section_id_map[relevant_section]
            
            # Получаем результаты поиска с реранкингом
            search_results = self.search_similar_sections(
                question, 
                top_k=max(k_values), 
                first_stage_k=first_stage_k
            )
            retrieved_ids = [result['id'] for result in search_results]
            
            # Рассчитываем метрики для текущего запроса
            query_metrics = {}
            for k in k_values:
                query_metrics[f'Recall@{k}'] = self.metrics_calculator.calculate_recall_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'Precision@{k}'] = self.metrics_calculator.calculate_precision_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'MRR@{k}'] = self.metrics_calculator.calculate_mrr_at_k([relevant_id], retrieved_ids, k)
                query_metrics[f'nDCG@{k}'] = self.metrics_calculator.calculate_ndcg_at_k([relevant_id], retrieved_ids, k)
            
            all_metrics.append(query_metrics)
        
        # Вычисляем средние метрики
        average_metrics = self.metrics_calculator.compute_average_metrics(all_metrics)
        
        # Добавляем информацию о модели и реранкере
        average_metrics['model_name'] = model_name
        average_metrics['reranker_name'] = reranker_name
        
        return average_metrics

In [16]:
# Функция для тестирования нескольких моделей и сравнения их результатов
def compare_models(data_path: str = 'qdrant_documentation_dataset.csv', k_values: List[int] = [1, 4, 6]):
    """Сравнивает различные модели и выводит их метрики."""
    # Создаем объект DocumentationQA
    qa_system = DocumentationQA()
    
    # Загружаем данные
    qa_system.load_data(data_path)
    
    # Список моделей для тестирования
    models = [
        SentenceTransformerModel('BAAI/bge-large-en-v1.5'),
        SentenceTransformerModel('intfloat/multilingual-e5-large'),
        BM25Model()
    ]
    
    # Оценка каждой модели
    results = []
    for model in models:
        print(f"\n===== Тестирование модели: {model.get_model_name()} =====")
        
        # Устанавливаем модель
        qa_system.set_embedding_model(model)
        
        # Инициализируем базу данных с новой моделью
        qa_system.initialize_database()
        
        # Оцениваем модель
        metrics = qa_system.evaluate_model(k_values=k_values)
        results.append(metrics)
        
        # Выводим результаты для текущей модели
        print("\nМетрики:")
        for metric_name, value in sorted(metrics.items()):
            if metric_name != 'model_name':
                print(f"{metric_name}: {value:.4f}")
    
    # Сравнительная таблица метрик
    print("\n===== Сравнение моделей =====")
    
    # Получаем все метрики
    all_metric_names = set()
    for result in results:
        all_metric_names.update([k for k in result.keys() if k != 'model_name'])
    
    # Форматируем таблицу
    metric_names_sorted = sorted(all_metric_names)
    model_names = [result['model_name'] for result in results]
    
    # Верхний заголовок таблицы
    header = f"{'Метрика':<15} | " + " | ".join(f"{name:<25}" for name in model_names)
    print(header)
    print("-" * len(header))
    
    # Строки таблицы
    for metric in metric_names_sorted:
        row = f"{metric:<15} | "
        for result in results:
            value = result.get(metric, 0.0)
            row += f"{value:.4f}{' ' * (25 - len(f'{value:.4f}'))}"
            row += " | "
        print(row[:-3])  # Убираем лишний разделитель в конце
    
    return results

In [17]:
def main():
    """Функция для вывода всех метрик по каждой модели с разными реранкерами."""
    # Определяем имена моделей и реранкеров (строки, а не объекты)
    embedding_model_names = [
        'BAAI/bge-base-en-v1.5',
        'intfloat/multilingual-e5-base',
        'BM25'  # Специальное значение для BM25
    ]
    
    reranker_names = [
        'None',  # Без реранкера
        'cross-encoder/ms-marco-MiniLM-L-12-v2',
        'BAAI/bge-reranker-base',
        'TinyBERT-MSMARCO',  # Специальное значение для TinyBERT
        'sentence-transformers/msmarco-distilbert-base-tas-b'
    ]
    
    # Создаем объект DocumentationQA
    qa_system = DocumentationQA()
    
    # Загружаем данные
    qa_system.load_data('qdrant_documentation_dataset.csv')
    
    # Создаем таблицу для итоговых результатов
    results_table = []
    
    # Перебираем модели эмбеддинга
    for model_name in embedding_model_names:
        # Создаем модель эмбеддинга на основе имени
        if model_name == 'BM25':
            embedding_model = BM25Model()
        else:
            try:
                # Очистка кэша PyTorch если доступно
                import torch
                if hasattr(torch, 'cuda') and torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                embedding_model = SentenceTransformerModel(model_name)
            except Exception as e:
                print(f"Ошибка при загрузке модели {model_name}: {e}")
                continue
        
        # Устанавливаем модель и инициализируем базу данных
        qa_system.set_embedding_model(embedding_model)
        try:
            qa_system.initialize_database()
        except Exception as e:
            print(f"Ошибка при инициализации базы данных с моделью {model_name}: {e}")
            continue
        
        # Перебираем реранкеры
        for reranker_name in reranker_names:
            print(f"\n===== Модель: {embedding_model.get_model_name()} + Реранкер: {reranker_name} =====")
            
            # Создаем реранкер на основе имени
            try:
                if reranker_name == 'None':
                    reranker = NoReranker()
                elif reranker_name == 'TinyBERT-MSMARCO':
                    reranker = TinyBertReranker()
                else:
                    # Очистка кэша PyTorch если доступно
                    if hasattr(torch, 'cuda') and torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    
                    reranker = CrossEncoderReranker(reranker_name)
                
                # Устанавливаем реранкер
                qa_system.set_reranker(reranker)
            except Exception as e:
                print(f"Ошибка при загрузке реранкера {reranker_name}: {e}")
                continue
            
            # Оцениваем модель с реранкером
            try:
                first_stage_k = 20  # Для реранкинга берем больше результатов
                metrics = qa_system.evaluate_model(k_values=[1, 4, 6], first_stage_k=first_stage_k)
                
                # Добавляем результат в таблицу
                result_row = {
                    'Embedding Model': embedding_model.get_model_name(),
                    'Reranker': reranker.get_reranker_name()
                }
                for metric_name, value in metrics.items():
                    if metric_name not in ('model_name', 'reranker_name'):
                        result_row[metric_name] = value
                
                results_table.append(result_row)
                
                # Выводим метрики по группам
                print("\nМетрики:")
                
                # Recall
                print("\nRecall@k:")
                for k in [1, 4, 6]:
                    print(f"  Recall@{k}: {metrics.get(f'Recall@{k}', 0.0):.4f}")
                
                # Precision
                print("\nPrecision@k:")
                for k in [1, 4, 6]:
                    print(f"  Precision@{k}: {metrics.get(f'Precision@{k}', 0.0):.4f}")
                
                # MRR
                print("\nMRR@k:")
                for k in [1, 4, 6]:
                    print(f"  MRR@{k}: {metrics.get(f'MRR@{k}', 0.0):.4f}")
                
                # nDCG
                print("\nnDCG@k:")
                for k in [1, 4, 6]:
                    print(f"  nDCG@{k}: {metrics.get(f'nDCG@{k}', 0.0):.4f}")
                    
            except Exception as e:
                print(f"Ошибка при оценке модели с реранкером {reranker_name}: {e}")
            
            # Очистка памяти после каждого теста
            if hasattr(torch, 'cuda') and torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            print("\n" + "="*50)
    
    # Вывод итоговой сравнительной таблицы
    print("\n===== Сводная таблица результатов =====")
    
    # Вывод заголовка таблицы
    if results_table:
        headers = ["Embedding Model", "Reranker"]
        for k in [1, 4, 6]:
            for metric in ["Recall", "Precision", "MRR", "nDCG"]:
                headers.append(f"{metric}@{k}")
        
        # Форматирование ширины столбцов - исправленная версия
        col_widths = []
        for header in headers:
            # Находим максимальную ширину для каждого столбца
            max_width = len(header)
            for row in results_table:
                value = row.get(header, "")
                value_str = f"{value:.4f}" if isinstance(value, float) else str(value)
                max_width = max(max_width, len(value_str))
            col_widths.append(max_width + 2)  # Добавляем отступ
        
        # Вывод заголовка
        header_row = "".join(f"{header:{width}}" for header, width in zip(headers, col_widths))
        print(header_row)
        print("-" * sum(col_widths))
        
        # Вывод данных
        for row in results_table:
            row_str = ""
            for i, header in enumerate(headers):
                value = row.get(header, "")
                if isinstance(value, float):
                    row_str += f"{value:.4f}{' ' * (col_widths[i] - len(f'{value:.4f}'))}"
                else:
                    row_str += f"{value}{' ' * (col_widths[i] - len(str(value)))}"
            print(row_str)
    
    # Находим лучшую комбинацию по nDCG@6
    if results_table:
        best_result = max(results_table, key=lambda x: x.get('nDCG@6', 0))
        print("\n===== Лучшая комбинация =====")
        print(f"Embedding: {best_result['Embedding Model']}")
        print(f"Reranker: {best_result['Reranker']}")
        print(f"nDCG@6: {best_result.get('nDCG@6', 0):.4f}")
        print(f"MRR@6: {best_result.get('MRR@6', 0):.4f}")
        print(f"Recall@6: {best_result.get('Recall@6', 0):.4f}")
        print(f"Precision@6: {best_result.get('Precision@6', 0):.4f}")
        
    # Сохраняем результаты в файл
    try:
        import pandas as pd
        pd.DataFrame(results_table).to_csv('final_results.csv', index=False)
        print("Финальные результаты сохранены в final_results.csv")
    except Exception as e:
        print(f"Ошибка при сохранении финальных результатов: {e}")
    

if __name__ == "__main__":
    main()

Загружено 328 записей из датасета.
Найдено 121 уникальных чанков для индексации.
Вычисление эмбеддингов для батча 1/2
Вычисление эмбеддингов для батча 2/2
База данных успешно инициализирована с моделью SentenceTransformer-bge-base-en-v1.5.

===== Модель: SentenceTransformer-bge-base-en-v1.5 + Реранкер: None =====
Оценка модели: SentenceTransformer-bge-base-en-v1.5 с реранкером: No-Reranker


Оценка модели:   0%|          | 0/328 [00:00<?, ?it/s]C:\Users\sekho\AppData\Local\Temp\ipykernel_25372\1846738923.py:73: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(
Оценка модели: 100%|██████████| 328/328 [00:13<00:00, 24.95it/s]



Метрики:

Recall@k:
  Recall@1: 0.6433
  Recall@4: 0.8811
  Recall@6: 0.9055

Precision@k:
  Precision@1: 0.6433
  Precision@4: 0.2203
  Precision@6: 0.1509

MRR@k:
  MRR@1: 0.6433
  MRR@4: 0.7409
  MRR@6: 0.7455

nDCG@k:
  nDCG@1: 0.6433
  nDCG@4: 0.7764
  nDCG@6: 0.7856


===== Модель: SentenceTransformer-bge-base-en-v1.5 + Реранкер: cross-encoder/ms-marco-MiniLM-L-12-v2 =====
Оценка модели: SentenceTransformer-bge-base-en-v1.5 с реранкером: CrossEncoder-ms-marco-MiniLM-L-12-v2


Оценка модели: 100%|██████████| 328/328 [11:16<00:00,  2.06s/it]



Метрики:

Recall@k:
  Recall@1: 0.7104
  Recall@4: 0.8780
  Recall@6: 0.9024

Precision@k:
  Precision@1: 0.7104
  Precision@4: 0.2195
  Precision@6: 0.1504

MRR@k:
  MRR@1: 0.7104
  MRR@4: 0.7790
  MRR@6: 0.7836

nDCG@k:
  nDCG@1: 0.7104
  nDCG@4: 0.8041
  nDCG@6: 0.8133


===== Модель: SentenceTransformer-bge-base-en-v1.5 + Реранкер: BAAI/bge-reranker-base =====
Оценка модели: SentenceTransformer-bge-base-en-v1.5 с реранкером: CrossEncoder-bge-reranker-base


...


Метрики:

Recall@k:
  Recall@1: 0.0030
  Recall@4: 0.0152
  Recall@6: 0.0274

Precision@k:
  Precision@1: 0.0030
  Precision@4: 0.0038
  Precision@6: 0.0046

MRR@k:
  MRR@1: 0.0030
  MRR@4: 0.0071
  MRR@6: 0.0093

nDCG@k:
  nDCG@1: 0.0030
  nDCG@4: 0.0091
  nDCG@6: 0.0137


===== Модель: BM25 + Реранкер: sentence-transformers/msmarco-distilbert-base-tas-b =====


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-tas-b and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Оценка модели: BM25 с реранкером: CrossEncoder-msmarco-distilbert-base-tas-b


Оценка модели: 100%|██████████| 328/328 [11:20<00:00,  2.07s/it]


Метрики:

Recall@k:
  Recall@1: 0.0396
  Recall@4: 0.1433
  Recall@6: 0.2317

Precision@k:
  Precision@1: 0.0396
  Precision@4: 0.0358
  Precision@6: 0.0386

MRR@k:
  MRR@1: 0.0396
  MRR@4: 0.0770
  MRR@6: 0.0932

nDCG@k:
  nDCG@1: 0.0396
  nDCG@4: 0.0935
  nDCG@6: 0.1264


===== Сводная таблица результатов =====
Embedding Model                           Reranker                                    Recall@1  Precision@1  MRR@1   nDCG@1  Recall@4  Precision@4  MRR@4   nDCG@4  Recall@6  Precision@6  MRR@6   nDCG@6  
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
SentenceTransformer-bge-base-en-v1.5      No-Reranker                                 0.6433    0.6433       0.6433  0.6433  0.8811    0.2203       0.7409  0.7764  0.9055    0.1509       0.7455  0.7856  
SentenceTransformer-bge-base-en-v1.5      CrossEncoder-ms-marco-MiniLM-L

Наилучшая комбинация - модель SentenceTransformer('BAAI/bge-base-en-v1.5') с реранкером cross-encoder/ms-marco-MiniLM-L-12-v2. Однако вижу большой разрыв между Recall@1 и Recall@4. Возможно будет лучше выбрать 2 или три фрагмента. Расчитаю дополнительные метрики для этой комбинации.

In [18]:
# Вычисление дополнительных метрик @2 и @3 для конкретной комбинации модели и реранкера
def calculate_additional_metrics():
    # Создаем объекты модели и реранкера
    embedding_model = SentenceTransformerModel('BAAI/bge-base-en-v1.5')
    reranker = CrossEncoderReranker('cross-encoder/ms-marco-MiniLM-L-12-v2')
    
    # Создаем QA систему с выбранной моделью и реранкером
    qa_system = DocumentationQA(embedding_model=embedding_model, reranker=reranker)
    
    # Загружаем данные
    qa_system.load_data('qdrant_documentation_dataset.csv')
    
    # Инициализируем базу данных (если она еще не инициализирована)
    if not qa_system.is_initialized:
        qa_system.initialize_database()
    
    # Метрики будем считать только для k=2 и k=3
    k_values = [2, 3]
    first_stage_k = 20  # Для реранкинга берем больше результатов
    
    # Получаем все метрики
    all_metrics = []
    model_name = embedding_model.get_model_name()
    reranker_name = reranker.get_reranker_name()
    
    print(f"Расчет дополнительных метрик для: {model_name} с реранкером: {reranker_name}")
    
    # Перебираем все вопросы в датасете
    for idx, row in tqdm(qa_system.df.iterrows(), total=len(qa_system.df), desc="Расчет метрик @2 и @3"):
        question = row['question']
        relevant_section = row['section_content']
        relevant_id = qa_system.section_id_map[relevant_section]
        
        # Получаем результаты поиска с реранкингом
        search_results = qa_system.search_similar_sections(
            question, 
            top_k=max(k_values), 
            first_stage_k=first_stage_k
        )
        retrieved_ids = [result['id'] for result in search_results]
        
        # Рассчитываем метрики для текущего запроса
        query_metrics = {}
        for k in k_values:
            query_metrics[f'Recall@{k}'] = qa_system.metrics_calculator.calculate_recall_at_k(
                [relevant_id], retrieved_ids, k)
            query_metrics[f'Precision@{k}'] = qa_system.metrics_calculator.calculate_precision_at_k(
                [relevant_id], retrieved_ids, k)
            query_metrics[f'MRR@{k}'] = qa_system.metrics_calculator.calculate_mrr_at_k(
                [relevant_id], retrieved_ids, k)
            query_metrics[f'nDCG@{k}'] = qa_system.metrics_calculator.calculate_ndcg_at_k(
                [relevant_id], retrieved_ids, k)
        
        all_metrics.append(query_metrics)
    
    # Вычисляем средние метрики
    average_metrics = qa_system.metrics_calculator.compute_average_metrics(all_metrics)
    
    # Выводим метрики
    print("\nДополнительные метрики:")
    for k in k_values:
        print(f"\nМетрики для k={k}:")
        print(f"  Recall@{k}: {average_metrics.get(f'Recall@{k}', 0.0):.4f}")
        print(f"  Precision@{k}: {average_metrics.get(f'Precision@{k}', 0.0):.4f}")
        print(f"  MRR@{k}: {average_metrics.get(f'MRR@{k}', 0.0):.4f}")
        print(f"  nDCG@{k}: {average_metrics.get(f'nDCG@{k}', 0.0):.4f}")
    
    return average_metrics

# Выполняем расчет
additional_metrics = calculate_additional_metrics()

Загружено 328 записей из датасета.
Найдено 121 уникальных чанков для индексации.
Вычисление эмбеддингов для батча 1/2
Вычисление эмбеддингов для батча 2/2
База данных успешно инициализирована с моделью SentenceTransformer-bge-base-en-v1.5.
Расчет дополнительных метрик для: SentenceTransformer-bge-base-en-v1.5 с реранкером: CrossEncoder-ms-marco-MiniLM-L-12-v2


Расчет метрик @2 и @3:   0%|          | 0/328 [00:00<?, ?it/s]C:\Users\sekho\AppData\Local\Temp\ipykernel_25372\1846738923.py:73: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self.client.search(
Расчет метрик @2 и @3: 100%|██████████| 328/328 [15:00<00:00,  2.74s/it]


Дополнительные метрики:

Метрики для k=2:
  Recall@2: 0.7988
  Precision@2: 0.3994
  MRR@2: 0.7546
  nDCG@2: 0.7661

Метрики для k=3:
  Recall@3: 0.8537
  Precision@3: 0.2846
  MRR@3: 0.7729
  nDCG@3: 0.7936


Итого, по Recall@3 выберем три фрагмента.